In [23]:
# Загрузка библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
from sklearn import datasets # для импорта данных
import seaborn as sns # для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков

# Линейная регрессия

Это модель с помощью которой мы описываем зависимость целевой переменной от факторов на обучающей выборке.

Рассмотрим классический датасет для обучения линейной регрессии — Boston Housing. В нём собраны усреднённые данные по стоимости недвижимости в 506 районах Бостона.

Целевой переменной будет <span style="color:red;">PRICE</span> — это, в некотором смысле, типичная (медианная) стоимость дома в районе.

Для примера возьмём в качестве регрессоров уровень преступности <span style="color:blue;">(CRIM)</span> и среднее количество комнат в доме <span style="color:blue;">(RM)</span>.

<img src='Images/ml_1.png'>

Запишем нашу модель:

$y = w_0 + w_1 \cdot x_1 + w_2 \cdot x_2$

Для наглядности обозначим:

$y = w_0 + w_1 \cdot CRIM + w_2 \cdot RM$

Составим матрицу регрессоров:

$A =
\begin{pmatrix}
1 & CRIM_1 & RM_1 \\
1 & CRIM_2 & RM_2 \\
\vdots & \vdots & \vdots \\
1 & CRIM_N & RM_N
\end{pmatrix}$

В нашем случае $N = 506$, а $k = 2$. Размерность матрицы $A$ будет равна:

$\dim A = (506, 3).$

Далее мы применяем формулу для вычисления оценок коэффициентов:

$\hat{w} = (A^T A)^{-1} A^T \vec{y}$

Вычисления к этой задаче мы сделаем в Python ниже, а пока приведём конечный результат. Если сократить запись до двух знаков после точки, получим следующие коэффициенты:

$\hat{w} = (-29.3, -0.26, 8.4)^T$

То есть:

$w_0 = -29.3$

$w_1 = -0.26$

$w_2 = 8.4$

Мы можем переписать нашу модель для прогноза:

$\hat{y} = -29.3 - 0.26 \cdot CRIM + 8.4 \cdot RM$

Теперь, если у нас появятся новые наблюдения, то есть ещё один небольшой район с уровнем преступности $0.1$ на душу населения и средним количеством комнат в доме, равным $8$, мы сможем сделать прогноз на типичную стоимость дома в этом районе — $37$ тысяч долларов:

$CRIM_{NEW} = 0.1$

$RM_{NEW} = 8$

$\hat{y}_{NEW} = -29.3 - 0.26 \cdot 0.1 + 8.4 \cdot 8 \approx 37$

# Решение на python

In [24]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
# 'header=None' указывает, что в файле нет строки заголовков, и мы задаем имена столбцов вручную
# 'delimiter=r"\s+"' указывает на то, что разделителем в файле являются пробелы (или любые пробелы)
# 'names=column_names' задаёт имена столбцов из списка 'column_names'
boston_data = pd.read_csv('Data/housing.csv', header=None, delimiter=r"\s+", names=column_names)

# Выводим первые пять строк DataFrame для просмотра загруженных данных
display(boston_data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Формируем матрицу A из столбца единиц и факторов CRIM и RM, а также вектор целевой переменной y:

In [25]:
# Составляем матрицу А (X) и вектор целевой переменной
CRIM = boston_data['CRIM']
RM = boston_data['RM']

A = np.column_stack((np.ones(506), CRIM, RM))
y = boston_data[['PRICE']]
display(A)

array([[1.0000e+00, 6.3200e-03, 6.5750e+00],
       [1.0000e+00, 2.7310e-02, 6.4210e+00],
       [1.0000e+00, 2.7290e-02, 7.1850e+00],
       ...,
       [1.0000e+00, 6.0760e-02, 6.9760e+00],
       [1.0000e+00, 1.0959e-01, 6.7940e+00],
       [1.0000e+00, 4.7410e-02, 6.0300e+00]], shape=(506, 3))

In [26]:
# Размерность матрицы A
print(A.shape)

(506, 3)


Теперь нам ничего не мешает вычислить оценку вектора коэффициентов $w$ по выведенной нами формуле МНК

$\hat{w} = (A^T A)^{-1} A^T \vec{y}$:

In [27]:
# вычислим OLS-оценку для коэффициентов
w_hat = np.linalg.inv(A.T @ A) @ A.T @ y

print(w_hat)

       PRICE
0 -29.244719
1  -0.264913
2   8.391068


Теперь составим прогноз нашей модели:

In [28]:
# добавились новые данные:
CRIM_new = 0.1
RM_new = 8

# делаем прогноз типичной стоимости дома
PRICE_new = w_hat.iloc[0] + w_hat.iloc[1] * CRIM_new + w_hat.iloc[2] * RM_new

print(PRICE_new)

PRICE    37.857335
dtype: float64


Для удобства дальнейшего использования оформим характеристики нового наблюдения в виде матрицы размером 

(3, 1):

In [29]:
# короткий способ сделать прогноз
new = np.array([[1, CRIM_new, RM_new]])
print(f'prediction: {(new @ w_hat).values}')

prediction: [[37.85733519]]


Мы уже знаем, что алгоритм построения модели линейной регрессии по МНК реализован в классе **LinearRegression**, находящемся в модуле **sklearn.linear_model**. Для вычисления коэффициентов (обучения модели) нам достаточно передать в метод **fit()** нашу матрицу с наблюдениями и вектор целевой переменной, а для построения прогноза — вызвать метод **predict()**:

Примечание: Здесь при создании объекта класса LinearRegression мы указали **fit_intercept=False**, так как в нашей матрице наблюдений  уже присутствует столбец с единицами для умножения на свободный член . Его повторное добавление не имеет смысла.

In [30]:
from sklearn.linear_model import LinearRegression

# Создаем модель линейной регрессии из библиотеки
model = LinearRegression(fit_intercept=False)

# Вычисляем коэффициенты
model.fit(A, y)
print(f'w_hat: {model.coef_}')

# Новые данные и прогноз по ним
new_prediction = model.predict(new)
print(f'prediction: {new_prediction}')

w_hat: [[-29.24471945  -0.26491325   8.39106825]]
prediction: [[37.85733519]]


Сделайте прогноз типичной стоимости (в тыс. долларов) дома в городе с уровнем преступности CRIM = 0.2 и средним количеством комнат в доме RM = 6. В качестве модели используйте линейную регрессию, оценка вектора коэффициентов которой равна:

$\hat{w} = (-29.3, -0.26, 8.4)$

Ответ округлите до целого числа.

In [31]:
# добавились новые данные:
CRIM_new = 0.2
RM_new = 6

new = np.array([[1, CRIM_new, RM_new]])

# Новые данные и прогноз по ним
new_prediction = model.predict(new)
print(f'prediction: {round(new_prediction[0][0])}')

prediction: 21


# Проблемы в классической МНК-модели

Давайте посмотрим, что «скажет» Python, если мы попробуем построить модель линейной регрессии на вырожденной матрице наблюдений, используя классическую формулу линейной регрессии.

In [32]:
# создадим вырожденную матрицу А
A = np.array([
    [1, 1, 1, 1], 
    [2, 1, 1, 2], 
    [-2, -1, -1, -2]]
).T

y = np.array([1, 2, 5, 1])

# вычислим OLS-оценку для коэффициентов
w_hat = np.linalg.inv(A.T @ A) @ A.T @ y
print(w_hat)

LinAlgError: Singular matrix

Как и ожидалось, мы получили ошибку, говорящую о том, что матрица $A^T A$ — сингулярная (вырожденная), а значит обратить её не получится. Что и требовалось доказать — с математикой всё сходится.

Попробуем обучить модель линейной регрессии LinearRegression из модуля sklearn, используя нашу вырожденную матрицу A:

In [61]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# вычисляем коэффициенты регрессии
model.fit(A, y)
print(f'w_hat = {model.coef_}')

w_hat = [ 6.   -1.25  1.25]


Никакой ошибки не возникло! Более того, у нас даже получились вполне адекватные оценки коэффициентов линейной регрессии $\hat{w}$. 

Дело в том, что в реализации линейной регрессии в sklearn предусмотрена борьба с плохо определёнными (близкими к вырожденным и вырожденными) матрицами. 

Для этого используется метод под названием **сингулярное разложение (SVD)**.

Суть метода заключается в том, что в OLS-формуле мы на самом деле используем не саму матрицу A, а её диагональное представление из сингулярного разложения, которое гарантированно является невырожденным.

Важно: В дальнейшем мы увидим, что доверять значениями полученным с помощью SVD лучше не стоит: возможна такая ситуация, при которой коэффициенты при линейно зависимых факторах, которые получаются в результате применения линейной регрессии через сингулярное разложение, могут получиться слишком большими по модулю. Они могут измеряться миллионами, миллиардами и более высокими порядками, что не будет иметь отношения к действительности. Такие коэффициенты не подлежат интерпретации.

Примечание. На самом деле сингулярное разложение зашито в функцию **np.linalg.lstsq()**, которая позволяет в одну строку построить модель линейной регрессии по МНК:

In [62]:
# классическая OLS-регрессия в numpy с возможностью получения решения даже для вырожденных матриц
np.linalg.lstsq(A, y, rcond=None)

(array([ 6.  , -1.25,  1.25]),
 array([], dtype=float64),
 np.int32(2),
 array([4.86435029, 0.58146041, 0.        ]))

Функция возвращает четыре значения:

- вектор рассчитанных коэффициентов линейной регрессии;
- сумму квадратов ошибок, MSE (она не считается, если ранг матрицы A меньше числа неизвестных, как в нашем случае);
- ранг матрицы A;
- вектор из сингулярных значений, которые как раз и оберегают нас от ошибки.

Обратите внимание, что мы получили те же коэффициенты, что и с помощью sklearn. При этом ранг матрицы A равен 2, что меньше количества неизвестных коэффициентов. Это ожидаемо говорит о вырожденности матрицы A и, как следствие, матрицы $A^T A$.

In [63]:
A = np.array([[1,-1,0],
             [1,1,2],
             [0,0,0],
             [2,0,2]]
             ).T

y = np.array([1,2,1,2])

w_hat = np.linalg.inv(A.T @ A) @ A.T @ y
print(w_hat)

LinAlgError: Singular matrix

# Стандартизация векторов

In [11]:
boston_data[['CHAS', 'LSTAT', 'CRIM', 'RM']].describe()

,CHAS,LSTAT,CRIM,RM
count,506.000000,506.000000,506.000000,506.000000
mean,0.069170,12.653063,3.613524,6.284634
std,0.253994,7.141062,8.601545,0.702617
min,0.000000,1.730000,0.006320,3.561000
25%,0.000000,6.950000,0.082045,5.885500
50%,0.000000,11.360000,0.256510,6.208500
75%,0.000000,16.955000,3.677083,6.623500
max,1.000000,37.970000,88.976200,8.780000


Видим, что каждый из признаков измеряется в различных единицах и изменяется в различных диапазонах: например, CHAS лежит в диапазоне от 0 до 1, а вот CRIM — в диапазоне от 0.006 до 88.976.

Рассмотрим модель линейной регрессии по МНК без стандартизации. Помним, что необходимо добавить столбец из единиц:

In [12]:
# составляем матрицу наблюдений и вектор целевой переменной
A = np.column_stack((np.ones(506), boston_data[['CHAS', 'LSTAT', 'CRIM', 'RM']]))
y = boston_data[['PRICE']]

# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat = np.linalg.inv(A.T @ A) @ A.T @ y
print(w_hat)

      PRICE
0 -1.920525
1  3.997559
2 -0.582402
3 -0.097394
4  5.075542


Давайте вспомним интерпретацию коэффициентов построенной модели линейной регрессии, которую мы изучали в модуле. Значение коэффициента $\hat{w_i}$ означает, на сколько в среднем изменится медианная цена (в тысячах долларов) при увеличении $x_1$ на 1.

Например, если количество низкостатусного населения (LSTAT) увеличится на 1 %, то медианная цена домов в районе (в среднем) упадёт на 0.6 тысяч долларов. А если среднее количество комнат (RM) в районе станет больше на 1, то медианная стоимость домов в районе (в среднем) увеличится на 5 тысяч долларов.

---
Тут в голову может прийти мысль: судя по значению коэффициентов, количество комнат (RM) оказывает на стоимость жилья большее влияние, чем процент низкостатусного населения (LSTAT). Однако такой вывод будет ошибочным. Мы не учитываем, что признаки, а значит и коэффициенты линейной регрессии, лежат в разных масштабах. Чтобы говорить о важности влияния признаков на модель, нужно строить её на **стандартизированных данных**.

Помним, что для построения стандартизированной линейной регрессии нам не нужен вектор свободных коэффициентов, а значит и столбец из единиц тоже не понадобится.

Сначала центрируем векторы, которые находятся в столбцах матрицы A. Для этого вычтем среднее, вычисленное по строкам матрицы A в каждом столбце, с помощью метода mean(). Затем разделим результат на длины центрированных векторов, вычисленных с помощью функции linalg.norm().

Примечание: Для функции linalg.norm() обязательно необходимо указать параметр axis=0, так как по умолчанию норма считается для всей матрицы, а не для каждого столбца в отдельности.

In [13]:
# составляем матрицу наблюдений без дополнительного столбца из единиц
A = boston_data[['CHAS', 'LSTAT', 'CRIM', 'RM']]
y = boston_data[['PRICE']]

# стандартизируем векторы в столбцах матрицы A
A_cent = A - A.mean(axis=0)
A_st = A_cent / np.linalg.norm(A_cent, axis=0)
A_st.describe().round(2)

,CHAS,LSTAT,CRIM,RM
count,506.00,506.00,506.00,506.00
mean,-0.00,-0.00,-0.00,-0.00
std,0.04,0.04,0.04,0.04
min,-0.01,-0.07,-0.02,-0.17
25%,-0.01,-0.04,-0.02,-0.03
50%,-0.01,-0.01,-0.02,-0.00
75%,-0.01,0.03,0.00,0.02
max,0.16,0.16,0.44,0.16


Для получения стандартизированных коэффициентов нам также понадобится стандартизация целевой переменной y по тому же принципу:

In [14]:
# стандартизируем вектор целевой переменной
y_cent = y - y.mean(axis=0)
y_st = y_cent / np.linalg.norm(y_cent, axis=0)

# вычислим OLS-оценку для стандартизированных коэффициентов
w_hat_st = np.linalg.inv(A_st.T @ A_st) @ A_st.T @ y_st
print(w_hat_st)

      PRICE
0  0.110400
1 -0.452204
2 -0.091088
3  0.387748


При стандартизации коэффициента $w_0$ у нас больше нет (интерсепта).

### Важный вывод:

Для того чтобы проинтерпретировать оценки коэффициентов линейной регрессии (понять, каков будет прирост целевой переменной при изменении фактора на 1 условную единицу), нам достаточно построить линейную регрессию в обычном виде без стандартизации и получить обычный вектор $\hat{w}$.

Однако, чтобы корректно говорить о том, какой фактор оказывает на прогноз бОльшее влияние, необходимо рассматривать стандартизированную оценку вектора коэффициентов $w_st$.

Если взглянуть на матрицу Грама для стандартизированных факторов:

In [15]:
display(A_st.T @ A_st)

,CHAS,LSTAT,CRIM,RM
CHAS,1.000000,-0.053929,-0.055892,0.091251
LSTAT,-0.053929,1.000000,0.455621,-0.613808
CRIM,-0.055892,0.455621,1.000000,-0.219247
RM,0.091251,-0.613808,-0.219247,1.000000


На самом деле мы только что вычислили матрицу выборочных корреляций наших исходных факторов (матрица корреляций) по которой можно определить степени линейной зависимости между факторами.

Примечание: Матрицу корреляций можно получить только в том случае, если производить стандартизацию признаков как векторы (делить на длину центрированного вектора $x_st$). Другие способы стандартизации/нормализации признаков не превращают матрицу Грама в матрицу корреляций!

In [20]:
x = np.array([12,8]).T

x_cent = x - x.mean(axis=0)
y_st = x_cent / np.linalg.norm(x_cent, axis=0)
print(f'{y_st[0].round(3)}, {y_st[1].round(3)}')

0.707, -0.707


# Корреляционная матрица

In [34]:
# Пример векторов
x_1 = np.array([1,2,6])
x_2 = np.array([3000,1000,2000])

# Корреляция между векторами (numpy)
print(np.corrcoef(x_1, x_2))

[[ 1.         -0.18898224]
 [-0.18898224  1.        ]]


# Практика

Вычислите коэффициент корреляции между векторами v и u.

Ответ округлите до двух знаков после точки-разделителя.

In [37]:
v = np.array([5,1,2]).T
u = np.array([4,2,8]).T

print(f'Корреляция (v, u) = \n{np.corrcoef(v, u).round(2)}')

Корреляция (v, u) = 
[[1.   0.05]
 [0.05 1.  ]]


Составьте корреляционную матрицу для системы векторов:

x_1, x_2, x_3

1. Чему равен ранг полученной корреляционной матрицы?

In [53]:
x_1 = np.array([5.1,1.8,2.1,10.3,12.1,12.6]).T
x_2 = np.array([10.2,3.7,4.1,20.5,24.2,24.1]).T
x_3 = np.array([2.5,0.9,1.1,5.1,6.1,6.3]).T

C_corr = np.corrcoef([x_1, x_2, x_3])
print(C_corr)
print('Определитель матрицы (C) = \n{:.7f}'.format(np.linalg.det(C_corr)))
print(f'Ранг матрицы (C) = {np.linalg.matrix_rank(C_corr)}')

[[1.         0.99925473 0.99983661]
 [0.99925473 1.         0.99906626]
 [0.99983661 0.99906626 1.        ]]
Определитель матрицы (C) = 
0.0000005
Ранг матрицы (C) = 3
